<a href="https://colab.research.google.com/github/ZeyadSabbah/TrivagoRecommenderSystem/blob/master/FeatureEngineeringMore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

## Mounting to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem

/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem


## Loading Libraries & Datasets

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt
from datetime import datetime
import re
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA

In [0]:
item_metadata_filepath = './Datasets/raw_data/item_metadata.csv'
item_metadata = pd.read_csv(item_metadata_filepath)

train_filepath = './Datasets/raw_data/Sets/train.csv'
train = pd.read_csv(train_filepath)

In [0]:
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


## Train More

What differs the most is the Final Clickout, but what if the customer wanted to book more than one hotel in the same country as they are going to have a long trip, so for the same session and different values of the cities the items are being looked up by the user, there is a high chance that the user is satisfies with the final item they selected in the each city. Data used can be increased by considering the final clickout in the other cities besides the final city.

In [5]:
trainMoreFClickout = train[train.action_type=='clickout item'].groupby('session_id')
trainMoreFClickout = (trainMoreFClickout.city.nunique()>1).to_frame().rename(columns={'city':'MoreFClickout'}).reset_index()
trainMoreFClickout = trainMoreFClickout[trainMoreFClickout.MoreFClickout==True]
trainMoreFClickout = train[train.session_id.isin(trainMoreFClickout.session_id.tolist())]
MoreFClickout = trainMoreFClickout[trainMoreFClickout.action_type=='clickout item'].groupby(['session_id', 'city']).tail(1)
MoreFClickout.head(3)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
947,1R2O77QTOU5A,484dfe03115ba,1541035374,1,clickout item,104770,BR,"Guarulhos, Brazil",desktop,NaN,152560|2454130|104651|104770|104806|104514|142...,42|34|113|54|103|44|43|41|42|45|51|46|20|51|43...
956,1R2O77QTOU5A,484dfe03115ba,1541036981,10,clickout item,104712,BR,"São Paulo, Brazil",desktop,NaN,104806|478461|6478386|5049198|104651|152560|24...,103|51|41|70|113|42|34|44|42|37|51|42|29|18|46...
2093,4GVU9AT4RVBW,b42f4cd99975e,1541106952,1,clickout item,42161,FR,"Amsterdam, Netherlands",mobile,NaN,921237|42161|42125|9051714|2775026|8435844|421...,89|69|110|126|75|98|61|84|52|96|80|88|57|84|14...


In [6]:
#changening session_id
c = MoreFClickout.groupby('session_id').cumcount()
c = c.replace(0, '').astype(str)
MoreFClickout['session_id'] += c
MoreFClickout.head(3)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
947,1R2O77QTOU5A,484dfe03115ba,1541035374,1,clickout item,104770,BR,"Guarulhos, Brazil",desktop,NaN,152560|2454130|104651|104770|104806|104514|142...,42|34|113|54|103|44|43|41|42|45|51|46|20|51|43...
956,1R2O77QTOU5A,484dfe03115ba1,1541036981,10,clickout item,104712,BR,"São Paulo, Brazil",desktop,NaN,104806|478461|6478386|5049198|104651|152560|24...,103|51|41|70|113|42|34|44|42|37|51|42|29|18|46...
2093,4GVU9AT4RVBW,b42f4cd99975e,1541106952,1,clickout item,42161,FR,"Amsterdam, Netherlands",mobile,NaN,921237|42161|42125|9051714|2775026|8435844|421...,89|69|110|126|75|98|61|84|52|96|80|88|57|84|14...


In [7]:
MoreFClickout[['user_id', 'session_id', 'city']]

,user_id,session_id,city
947,1R2O77QTOU5A,484dfe03115ba,"Guarulhos, Brazil"
956,1R2O77QTOU5A,484dfe03115ba1,"São Paulo, Brazil"
2093,4GVU9AT4RVBW,b42f4cd99975e,"Amsterdam, Netherlands"
2100,4GVU9AT4RVBW,b42f4cd99975e1,"Utrecht, Netherlands"
2349,52IZXNUIF0NL,60f16cf8330b7,"Tokyo, Japan"
...,...,...,...
10665708,Y813RRM1N7P2,219a868ad5a611,"Santiago, Chile"
10665804,YIRGODEF6SO6,b8c521849e3c2,"Dortmund, Germany"
10665826,YIRGODEF6SO6,b8c521849e3c21,"Luebeck, Germany"
10665866,Z6W0KZ1LCB8H,a4c049a08e980,"Murray Bridge, Australia"


In [7]:
trainMoreFClickout = trainMoreFClickout.rename(columns={'session_id':'old_session_id'})\
                     .merge(MoreFClickout[['user_id', 'session_id', 'city']], on=['user_id', 'city'], how='left')\
                     .drop(columns='old_session_id')
cols = train.columns.tolist()
trainMoreFClickout = trainMoreFClickout[cols]
trainMoreFClickout

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,1R2O77QTOU5A,484dfe03115ba,1541035374,1,clickout item,104770,BR,"Guarulhos, Brazil",desktop,NaN,152560|2454130|104651|104770|104806|104514|142...,42|34|113|54|103|44|43|41|42|45|51|46|20|51|43...
1,1R2O77QTOU5A,484dfe03115ba1,1541036663,2,search for poi,International Airport São Paulo Guarulhos,BR,"São Paulo, Brazil",desktop,NaN,NaN,NaN
2,1R2O77QTOU5A,484dfe03115ba1,1541036711,3,clickout item,104712,BR,"São Paulo, Brazil",desktop,NaN,104806|478461|6478386|5049198|104651|152560|24...,103|51|41|70|113|42|34|44|42|37|51|42|29|18|46...
3,1R2O77QTOU5A,484dfe03115ba1,1541036775,4,clickout item,104770,BR,"São Paulo, Brazil",desktop,NaN,104806|478461|6478386|5049198|104651|152560|24...,103|51|41|70|113|42|34|44|42|37|51|42|29|18|46...
4,1R2O77QTOU5A,484dfe03115ba1,1541036970,5,interaction item rating,104712,BR,"São Paulo, Brazil",desktop,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1012576,Z6W0KZ1LCB8H,a4c049a08e9801,1541498764,42,interaction item image,4201464,AU,"Adelaide, Australia",mobile,NaN,NaN,NaN
1012577,Z6W0KZ1LCB8H,a4c049a08e9801,1541498764,43,interaction item image,4201464,AU,"Adelaide, Australia",mobile,NaN,NaN,NaN
1012578,Z6W0KZ1LCB8H,a4c049a08e9801,1541498774,44,interaction item image,4646574,AU,"Adelaide, Australia",mobile,NaN,NaN,NaN
1012579,Z6W0KZ1LCB8H,a4c049a08e9801,1541498774,45,interaction item image,4646574,AU,"Adelaide, Australia",mobile,NaN,NaN,NaN


In [9]:
# Concatenating trainMoreFClickout to train
trainMore = pd.concat([train, trainMoreFClickout]).drop_duplicates()
trainMore.head(2)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


## Important Dataframes Preparation

In engineer global features from the train dataset, which will be most of the cases around either clickouts or final click out examples (instances), two dataframes of them will be created.

In order to get representative global values to the items, duplications must be removed. Rows would not all be duplicated because of some attributes such as timestamp, reference, and step.

In [0]:
ClickoutDF = trainMore[trainMore.action_type=='clickout item']
FinalClickoutDF = trainMore[trainMore.action_type=='clickout item'].groupby('session_id').tail(1)
ClickoutUniqueDF = ClickoutDF.drop_duplicates(subset=['session_id', 'impressions'], keep='first')

## Applying transform_data Function

All the functions are saved in a class that can be imported and apply the function in just one line.

In [10]:
from data_transformation import data_transformation

TrainMoreData = data_transformation.transform_data(trainMore)
TrainMoreData

/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem/data_transformation.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_id['impressions'] = item_id['impressions'].apply(lambda x: x.split('|'))
/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem/data_transformation.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['prices'] = price['prices'].apply(lambda x: x.split('|'))
/content/drive/My Drive/Trivago/Project/TrivagoRecommenderSystem/data_transformation.py:44: Se

,session_id,item_id,price,item_rank,price_rank,clickout,session_duration,item_duration,item_session_duration,item_interactions,maximum_step,top_list,NumberOfProperties,NumberInImpressions,NumberInReferences,NumberAsClickout,NumberAsFinalClickout,FClickoutToImpressions,FClickoutToReferences,FClickoutToClickout,MeanRank,MeanPrice,MaxPrice,MinPrice,AveragePriceRank
0,aff3928535f48,55109,162,1,8,0,1025,0.0,0.000000,0.0,16,1,58.0,584.0,479.0,25.0,11.0,0.018836,0.022965,0.440000,12.239726,194.595890,1905.0,65.0,11.840753
1,aff3928535f48,129343,25,2,1,0,1025,0.0,0.000000,0.0,16,1,26.0,108.0,69.0,9.0,4.0,0.037037,0.057971,0.444444,11.083333,72.259259,388.0,18.0,10.166667
2,aff3928535f48,54824,150,3,15,0,1025,0.0,0.000000,0.0,16,1,42.0,443.0,386.0,61.0,32.0,0.072235,0.082902,0.524590,11.627540,107.672686,408.0,42.0,11.291196
3,aff3928535f48,2297972,143,4,5,0,1025,0.0,0.000000,0.0,16,1,38.0,17.0,23.0,2.0,0.0,0.000000,0.000000,0.000000,13.294118,136.000000,183.0,101.0,9.647059
4,aff3928535f48,109014,101,5,12,0,1025,0.0,0.000000,0.0,16,1,60.0,194.0,268.0,18.0,9.0,0.046392,0.033582,0.500000,14.443299,149.329897,628.0,70.0,11.262887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13192212,a4c049a08e9801,1425092,84,9,3,0,1037,0.0,0.000000,0.0,46,0,36.0,44.0,16.0,2.0,1.0,0.022727,0.062500,0.500000,14.931818,66.772727,156.0,54.0,11.500000
13192213,a4c049a08e9801,1627067,94,10,10,0,1037,13.0,0.012536,2.0,46,0,54.0,126.0,98.0,15.0,9.0,0.071429,0.091837,0.600000,12.349206,97.896825,216.0,59.0,11.031746
13192214,a4c049a08e9801,4201464,143,11,2,0,1037,152.0,0.146577,15.0,46,0,16.0,12.0,17.0,2.0,1.0,0.083333,0.058824,0.500000,15.416667,121.250000,158.0,92.0,8.000000
13192215,a4c049a08e9801,2106906,137,12,0,0,1037,0.0,0.000000,0.0,46,0,32.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,12.000000,137.000000,137.0,137.0,0.000000
